# Notebook 02: EDA

In this notebook I'm going to dive a bit deeper into the (now cleaned) dataset

In [1]:
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport
import altair as alt
import altair_data_server

from sklearn.feature_extraction.text import CountVectorizer

from src.data.load_data import Data

alt.data_transformers.enable("data_server")


DataTransformerRegistry.enable('data_server')

In [2]:
df = Data().load(clean=True)

In [3]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4814 entries, 0 to 5267
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          4814 non-null   datetime64[ns]
 1   year          4814 non-null   int64         
 2   month         4814 non-null   string        
 3   location      4814 non-null   string        
 4   country       4814 non-null   category      
 5   sector        4814 non-null   category      
 6   operator      4814 non-null   string        
 7   type          4814 non-null   string        
 8   aboard        4814 non-null   int64         
 9   fatalities    4814 non-null   int64         
 10  fatality_pct  4812 non-null   float64       
 11  ground        4814 non-null   int64         
 12  summary       4814 non-null   string        
dtypes: category(2), datetime64[ns](1), float64(1), int64(4), string(5)
memory usage: 2.9 MB


In [4]:
df.sample(10)

,date,year,month,location,country,sector,operator,type,aboard,fatalities,fatality_pct,ground,summary
2877,1976-09-02,1976,September,"Hawk Inlet, Alaska",United States,Commercial,Temsco Helicopter - Air Taxi,Hughes 369HS,4,3,0.750000,0,Crashed while climbing to cruise altitude. Fai...
1138,1951-07-12,1951,July,"Lyneham AFB, England",England,Military,Military - Royal Air Force,Vickers Valetta C-1,10,10,1.000000,0,One engine caught fire. The propellers of the ...
3887,1989-10-04,1989,October,"Perth, Ontario, Canada",Canada,Military,Military - U.S. Air Force,Boeing KC-135A,4,4,1.000000,0,A dry fuel pump led to the ignition of fuel v...
5151,2007-06-13,2007,June,"Selenge province, Mongolia",Mongolia,Commercial,Mongolian Defense Ministry,Mi-8,22,15,0.681818,0,"While en route to a forest fire, the helicopte..."
4291,1994-10-26,1994,October,"Near Cuito, Angola",Angola,Military,Military - Angolan Air Force,Mil Mi-8MTV (helicopter),22,22,1.000000,0,Shot down by rebel forces.
479,1938-07-27,1938,July,"Kisumu, Kenya",Kenya,Commercial,Imperial Airways,Armstrong-Withworth Atlanta,4,4,1.000000,0,Flew into a hillside soon after takeoff from K...
2499,1971-10-02,1971,October,"Near Aarsele, West Vlaanderen, Belgium",Belgium,Commercial,British European Airways,Vickers Vanguard 951,63,63,1.000000,0,While en route a distress message was received...
4467,1996-12-21,1996,December,"Rio Negro, Colombia",Colombia,Commercial,SELVA,Antonov An-32B,4,4,1.000000,0,The cargo plane crashed short of the runway wh...
2842,1975-09-11,1975,September,"Near Bahar Dar, Ethiopia",Ethiopia,Commercial,Ethiopian Airlines,Douglas C-47-DL,9,1,0.111111,0,The plane was flying at FL 115 under VFR in sc...
3179,1979-11-01,1979,November,"Big Trout Lake, Canada",Canada,Commercial,Austin Airways,De Havilland Dash-6,3,3,1.000000,0,Sruck the tower while trying to land in poor v...


## Data Cleaning

So you'll see we now have a cleaned up version of the raw data when I pass `clean = True` to the `load` method of the `Data` class. If you're interested, this is all implemented in `src/data/load_data.py`.

Basic summary of the cleaning steps:

* Dropped a few columns:
    * `Flight #` - I didn't really see any gainful insights coming from this one, essentially just random numbers generated by the airline
    * `Registration` - Similar thing. This is the aircraft's registration number e.g. G-EGHG.
    * `cn/In` - Same as registration, this refers to the aircraft's airframe ID or model number.
    * `Route` - Initially I wanted to keep this to see if any one route is particularly hazardous etc. but on some scratchpad analysis it seemed there were just too many distinct values to really learn anything.
    * `Time` - This is the one I'm most sad about losing. I strongly suspect that accidents at night time are more common than during the day. But there were lots of missing values and I couldn't get it to play nicely in my cleaning pipeline with pandas datetime parsing. It ended up parsing the date correctly but then because the time was `NaN` the whole thing would get cast to an object. If anyone knows a good way of handling this let me know!

* Did some basic formatting and tidying i.e. stripping any whitespace from text, dropped any remaining `NaNs`

* Extracted some features like `year`, `month`, `country`, whether it was a military, private, or commercial flight etc.

* Grouped some countries for example, the country extractor would return names of US states, so I wrote a method to group them under `United States`. Similarly grouped `USSR` and `Russia`

* And finally, reordered the columns to satisfy my OCD!

## EDA

Let's get into it!

I'm going to start with my favourite auto-data-magic-summariser... pandas-profiling. I usually do this at the start of an EDA and keep it around to refer back to

In [5]:
# ProfileReport(df)

Let's start by finding out the top 10 most dangerous countries to fly in...

In [6]:
(alt.Chart(df.groupby(by = "country")[["fatalities"]].sum().nlargest(10, ["fatalities"]).reset_index())
.mark_bar()
.encode(
    x = alt.X("country:N", title = "Country", sort = "-y",),
    y = alt.Y("fatalities:Q", title = "Total Fatalities"),
    tooltip = ["country", "fatalities"]
).properties(
    title = "Total Fatalities by Country (Top 10)",
    height = 500,
    width = 750
).configure_axisX(
    labelAngle = -40
))

alt.Chart(...)

Wow, the US does not come off well here. This could be real, however it could also be anything like:

* The data was collected from a source in the US enabling more easy data collection for domestic accidents
* The population of the US regularly travelling by air is likely to be high compared to the other countries here

In [7]:
month_order = [
    "January",
    "February",
    "March",
    "April",
    "May",
    "June",
    "July",
    "August",
    "September",
    "October",
    "November",
    "December",
]

alt.Chart(df).mark_bar().encode(
    x = alt.X("month:N", title = "Month", sort = month_order),
    y = alt.Y("sum(fatalities):Q", title = "Total Fatalities")
).properties(
    title = "Total Fatalities by Month",
    height = 500,
    width = 750
).configure_axisX(
    labelAngle = -40
)

alt.Chart(...)

In [8]:
# Get a word frequency list for summary
cv = CountVectorizer(stop_words = "english")

cv_fit = cv.fit_transform(df['summary'])


words = cv.get_feature_names()
counts = np.asarray(cv_fit.sum(axis = 0))

# Cast to dictionary of word: frequency
wordcount_dict = dict(zip(words, counts[0]))

# Make a dataframe from dict and sort by frequency
word_df = pd.DataFrame.from_dict(wordcount_dict, orient="index", columns = ["Count"]).reset_index().rename(columns = {"index": "Word"}).sort_values("Count", ascending = False)

In [9]:
word_df

,Word,Count
2313,crashed,3182
640,aircraft,2470
6411,plane,1843
6373,pilot,1284
3448,flight,1165
...,...,...
2551,denied,1
5774,neveu,1
2552,dennis,1
5772,nevada,1
